# Uniswap V3 position with hedge

The document contains calculations for the liquidity, tokens amounts and fees. The formulae may be found in the paper [paper](https://atiselsts.github.io/pdfs/uniswap-v3-liquidity-math.pdf) by Atis Elsts and in the original [whitepaper](https://uniswap.org/whitepaper-v3.pdf) for Uniswap V3. 

In [95]:
!pip install gql
!pip install plotly
!pip install requests

#for RequestsHTTPTransport
!pip install requests-toolbelt 

#Async Usage
!pip install asyncio
!pip install aiohttp

#Deribit
!pip install websockets

In [96]:
import pandas as pd
import numpy as np
import datetime as dt
from tqdm import tqdm
import requests
import json
import plotly.express as px
import matplotlib.pyplot as plt
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
%matplotlib inline

#Async Usage
import asyncio
from gql.transport.aiohttp import AIOHTTPTransport

import warnings
warnings.filterwarnings('ignore')

In [97]:
# connect to the Uniswap V3 Subgraph
uni_transport = RequestsHTTPTransport(
    url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3', #here we can use uniswap-v3
    verify = True,
    retries=3,  
)
client = Client(transport = uni_transport)

## Load Uniswap V3 pool data using TheGraph

In [98]:
data_pools = pd.DataFrame()

In [99]:
# get the set of pools available in the Uniswap V3 Subgraph

query = gql(
    """
query ($first: Int!, $skip: Int!) {
  pools (first: $first, skip: $skip){
    id
    volumeUSD
    token0{
        name
    }
    token1{
        name
    }
  }
}
"""
)

page_size = 1000
skip = 0

while True:
    if skip > 5000:
        break 
    vars = {"first": page_size, "skip": skip}
    result = client.execute(query, variable_values=vars)
    
    data = []
    for i in result['pools']:
        data.append([
            i['id'],
            i['volumeUSD'],
            i['token0']['name'],
            i['token1']['name']
        ])
        
    df = pd.DataFrame(data)
    df.columns = ['id','volumeUSD','name0','name1']
    df['Pair'] = df['name0'] + '-' + df['name1']
    df['volumeUSD'] = pd.to_numeric(df['volumeUSD'])     
        
    data_pools = data_pools.append(df)   
    skip += page_size
    
    if not result['pools']:
        break


In [100]:
# select the large pools with at least $1B in trading volume

data_pools = data_pools.reset_index()
data_pools['volumeUSD'] = data_pools['volumeUSD'].astype('int64')
large_pools = data_pools[ data_pools['volumeUSD'] > 1000000000]
large_pools

,index,id,volumeUSD,name0,name1,Pair
376,376,0x07f3d316630719f4fc69c152f397c150f0831071,1062468490,Euro Tether,Tether USD,Euro Tether-Tether USD
797,797,0x11b815efb8f581194ae79006d24e0d814b7697f6,60952061507,Wrapped Ether,Tether USD,Wrapped Ether-Tether USD
946,946,0x151ccb92bc1ed5c6d0f9adb5cec4763ceb66ac7f,2736862446,ETH 2x Flexible Leverage Index,Wrapped Ether,ETH 2x Flexible Leverage Index-Wrapped Ether
1115,115,0x18d96b617a3e5c42a2ada4bc5d1b48e223f17d0d,4133394213,USD Coin,Wrapped UST Token,USD Coin-Wrapped UST Token
1310,310,0x1c98562a2fab5af19d8fb3291a36ac3c618835d9,1517783177,Metis Token,Wrapped Ether,Metis Token-Wrapped Ether
1341,341,0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801,4326728340,Uniswap,Wrapped Ether,Uniswap-Wrapped Ether
1674,674,0x24ee2c6b9597f035088cda8575e9d5e15a84b9df,1019386402,Quant,Wrapped Ether,Quant-Wrapped Ether
1843,843,0x290a6a7460b308ee3f19023d2d00de604bcf5b42,4087849094,Matic Token,Wrapped Ether,Matic Token-Wrapped Ether
1864,864,0x298b7c5e0770d151e4c5cf6cca4dae3a3ffc8e27,1191192232,Magic Internet Money,USD Coin,Magic Internet Money-USD Coin
2105,105,0x2f62f2b4c5fcd7570a709dec05d68ea19c82a9ec,2148378566,SHIBA INU,Wrapped Ether,SHIBA INU-Wrapped Ether


In [101]:
fig = px.bar(large_pools, x = 'Pair', y = 'volumeUSD')
fig.show()

## Liquidity provision in Uniswap V3

USDC - WETH pool: 0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640

In [102]:
data_pool = pd.DataFrame()

# query for the USDC-WETH pool
query = gql(
    """
    query ($first: Int!, $skip: Int!) {
      pool(id: "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640") {
        poolHourData (first: $first, skip: $skip){
          periodStartUnix
          token0Price
          token1Price
          tick
          id
        }
      }
    }
    
    """
)   

page_size = 1000
skip = 0

while True:
    if skip > 2000:
        break 
    vars = {"first": page_size, "skip": skip}
    result = client.execute(query, variable_values=vars)
    
    data = []
    for i in result['pool']['poolHourData']:
        data.append([
            i['periodStartUnix'],
            i['token0Price'],
            i['token1Price'],
            i['tick'],
            i['id']
        ])
    
    df = []
    df = pd.DataFrame(data)

    df.columns = ['date','token0Price','token1Price','tick','id']

    df['token0Price'] = pd.to_numeric(df['token0Price'])
    df['token1Price'] = pd.to_numeric(df['token1Price'])
    df['datetime'] = 0

    df['USDC/WETH'] = pd.to_numeric(df['token0Price']/df['token1Price'])

    for i in range(len(df)):
        df['datetime'][i] = dt.datetime.fromtimestamp(df['date'][i]).strftime('%Y-%m-%d %H:%M:%S')
    
    data_pool = data_pool.append(df)
    skip += page_size    
    
    if not result['pool']:
        break

In [103]:
data_pool

,date,token0Price,token1Price,tick,id,datetime,USDC/WETH
0,1620248400,0.000000,0.000000,None,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640-450069,2021-05-06 00:00:00,NaN
1,1620252000,3402.729189,0.000294,194996,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640-450070,2021-05-06 01:00:00,1.157857e+07
2,1620255600,3496.203173,0.000286,194725,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640-450071,2021-05-06 02:00:00,1.222344e+07
3,1620259200,3480.198579,0.000287,194771,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640-450072,2021-05-06 03:00:00,1.211178e+07
4,1620262800,3503.486877,0.000285,194704,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640-450073,2021-05-06 04:00:00,1.227442e+07
...,...,...,...,...,...,...,...
995,1631030400,3574.064249,0.000280,194505,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640-453064,2021-09-07 19:00:00,1.277394e+07
996,1631034000,3419.354725,0.000292,194947,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640-453065,2021-09-07 20:00:00,1.169199e+07
997,1631037600,3451.361659,0.000290,194854,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640-453066,2021-09-07 21:00:00,1.191190e+07
998,1631041200,3415.906179,0.000293,194957,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640-453067,2021-09-07 22:00:00,1.166842e+07


To construct the pseudo delta neutral strategy we need to use perps. We'll take the data on ETH perps from the Binance API

In [104]:
# funding rates for ETH

data_funding =  pd.DataFrame()
start_month = [5,6,7,8,9,10,11]

for i in range(len(start_month)):
    url = 'https://fapi.binance.com/fapi/v1/fundingRate'
    symbol = 'ETHUSDT'
    interval = '8h' # 1-3-5-15-30m 1-2-4-6-8-12h 1-3d 1w 1M
    start = str(int(dt.datetime(2021,start_month[i],6).timestamp()*1000))
    end = str(int(dt.datetime(2022,12,31).timestamp()*1000))
    params = {'symbol': symbol, 'interval': interval, 'startTime': start, 'endTime': end}

    data = pd.DataFrame(json.loads(requests.get(url, params= params).text))

    data.index = [dt.datetime.fromtimestamp(x/1000.0) for x in data.fundingTime]
    data['datetime'] = [dt.datetime.fromtimestamp(x/1000.0).strftime('%Y-%m-%d %H:%M:%S') for x in data.fundingTime]
    data['fundingRate'] = pd.to_numeric(data['fundingRate'])
    
    # Gather the data into one dataframe
    data_funding = data_funding.append(data)

In [105]:
#drop dublicate dates
data_funding = data_funding.drop_duplicates()
data_funding

,symbol,fundingTime,fundingRate,datetime
2021-05-06 03:00:00.003,ETHUSDT,1620259200003,0.000682,2021-05-06 03:00:00
2021-05-06 11:00:00.001,ETHUSDT,1620288000001,0.000666,2021-05-06 11:00:00
2021-05-06 19:00:00.000,ETHUSDT,1620316800000,0.001109,2021-05-06 19:00:00
2021-05-07 03:00:00.000,ETHUSDT,1620345600000,0.000260,2021-05-07 03:00:00
2021-05-07 11:00:00.000,ETHUSDT,1620374400000,0.000491,2021-05-07 11:00:00
...,...,...,...,...
2021-12-07 19:00:00.000,ETHUSDT,1638892800000,0.000100,2021-12-07 19:00:00
2021-12-08 03:00:00.000,ETHUSDT,1638921600000,0.000100,2021-12-08 03:00:00
2021-12-08 11:00:00.018,ETHUSDT,1638950400018,0.000100,2021-12-08 11:00:00
2021-12-08 19:00:00.010,ETHUSDT,1638979200010,0.000100,2021-12-08 19:00:00


In [106]:
data_total = data_pool
data_total = data_total[['datetime','token0Price','token1Price','tick']]
data_total

,datetime,token0Price,token1Price,tick
0,2021-05-06 00:00:00,0.000000,0.000000,None
1,2021-05-06 01:00:00,3402.729189,0.000294,194996
2,2021-05-06 02:00:00,3496.203173,0.000286,194725
3,2021-05-06 03:00:00,3480.198579,0.000287,194771
4,2021-05-06 04:00:00,3503.486877,0.000285,194704
...,...,...,...,...
995,2021-09-07 19:00:00,3574.064249,0.000280,194505
996,2021-09-07 20:00:00,3419.354725,0.000292,194947
997,2021-09-07 21:00:00,3451.361659,0.000290,194854
998,2021-09-07 22:00:00,3415.906179,0.000293,194957


In [107]:
all_data = pd.merge(data_total, data_funding, how="left", on='datetime')
all_data['token0-token1'] = all_data['token0Price'] - all_data['token1Price']
all_data['token0_p'] = all_data['token0Price']
all_data['token1_p'] = all_data['token1Price']
all_data['token0_f'] = all_data['fundingRate']
all_data['token1_f'] = np.NaN
all_data = all_data[['datetime','token0-token1','token0_p','token1_p','token0_f','token1_f','tick',]]
all_data

,datetime,token0-token1,token0_p,token1_p,token0_f,token1_f,tick
0,2021-05-06 00:00:00,0.000000,0.000000,0.000000,NaN,NaN,None
1,2021-05-06 01:00:00,3402.728895,3402.729189,0.000294,NaN,NaN,194996
2,2021-05-06 02:00:00,3496.202887,3496.203173,0.000286,NaN,NaN,194725
3,2021-05-06 03:00:00,3480.198291,3480.198579,0.000287,0.000682,NaN,194771
4,2021-05-06 04:00:00,3503.486592,3503.486877,0.000285,NaN,NaN,194704
...,...,...,...,...,...,...,...
2995,2021-09-07 19:00:00,3574.063969,3574.064249,0.000280,0.000100,NaN,194505
2996,2021-09-07 20:00:00,3419.354433,3419.354725,0.000292,NaN,NaN,194947
2997,2021-09-07 21:00:00,3451.361369,3451.361659,0.000290,NaN,NaN,194854
2998,2021-09-07 22:00:00,3415.905886,3415.906179,0.000293,NaN,NaN,194957


Liquidity provision strategy: put capital into the Uniswap V3 pool and use perps to hedge the position. 

In [109]:
class UniPosition:
    def __init__(self, liquidity, tick_lower, tick_upper):
        self.liquidity = liquidity
        self.tick_lower = tick_lower
        self.tick_upper = tick_upper
        
    def get_tokens_amount(self, tick):
        tick_base = 1.0001
        sqrt_price = tick_base ** (tick / 2)
        sqrt_price_l = tick_base ** (self.tick_lower / 2)
        sqrt_price_u = tick_base ** (self.tick_upper / 2)
        if tick < self.tick_lower:
            token0_amount = self.liquidity * (1 / sqrt_price_l - 1 / sqrt_price_u)
            token1_amount = 0
        elif tick > self.tick_upper:
            token0_amount = 0
            token1_amount = self.liquidity * (sqrt_price_u - sqrt_price_l)
        else:
            token0_amount = self.liquidity * (1 / sqrt_price - 1 / sqrt_price_u)
            token1_amount = self.liquidity * (sqrt_price - sqrt_price_l)
        return token0_amount, token1_amount

In [110]:
class UniPosition2:
    def __init__(self, init_liquidity, init_pool_balance, init_tick, init_position_width):
        self.liquidity = init_liquidity
        self.position_width = init_position_width
        self.tick_lower = init_tick - init_position_width
        self.tick_upper = init_tick + init_position_width
        self.pool_balance = init_pool_balance

    
    def get_low_tick(self, vol_change):
        if pd.notna(vol_change):
            new_tick_lower = init_tick - round(self.position_width - (vol_change>1.05)*1000,0) #update the price range: if the vol grows we use a wider range
        else:
            new_tick_lower = init_tick - self.position_width
            
        self.tick_lower = new_tick_lower
        
    def get_upp_tick(self, vol_change):
        if pd.notna(vol_change):
            new_tick_upper = init_tick + round(self.position_width - (vol_change>1.05)*1000,0)
        else:
            new_tick_upper = init_tick + self.position_width
            
        self.tick_upper = new_tick_upper
    
    def calculate_liquidity(self, tick, vol_change, price0, price1):
        X_amount = self.pool_balance / 2 / price0
        Y_amount = self.pool_balance / 2 / price1
        tick_base = 1.0001
        if vol_change != 0:
            self.get_low_tick(vol_change)
            self.get_upp_tick(vol_change)
            
        sqrt_price = tick_base ** (tick / 2)
        sqrt_price_l = tick_base ** (self.tick_lower / 2)
        sqrt_price_u = tick_base ** (self.tick_upper / 2)

        if tick <= self.tick_lower:
            liq_x = (X_amount / (1 / sqrt_price_l - 1 / sqrt_price_u))
            liq_y = liq_x
            
        elif tick >= self.tick_upper:
            liq_y = (Y_amount / (sqrt_price_u - sqrt_price_l))
            liq_x = liq_y
        else:
            liq_x = (X_amount / (1 / sqrt_price - 1 / sqrt_price_u))
            liq_y = (Y_amount / (sqrt_price - sqrt_price_l))
            
        self.liquidity = min (liq_x,liq_y)
    
    def get_tokens_amount(self, tick, vol_change, price0, price1): #pool_balance_0
        tick_base = 1.0001
        sqrt_price = tick_base ** (tick / 2)
        
        sqrt_price_l = tick_base ** (self.tick_lower / 2)
        sqrt_price_u = tick_base ** (self.tick_upper / 2)
        
        if tick < self.tick_lower:
            token0_amount = self.liquidity * (1 / sqrt_price_l - 1 / sqrt_price_u)
            token1_amount = 0
        elif tick > self.tick_upper:
            token0_amount = 0
            token1_amount = self.liquidity * (sqrt_price_u - sqrt_price_l)
        else:
            token0_amount = self.liquidity * (1 / sqrt_price - 1 / sqrt_price_u)
            token1_amount = self.liquidity * (sqrt_price - sqrt_price_l)
        
        self.pool_balance = token0_amount * price0 + token1_amount * price1
        
        return token0_amount, token1_amount

In [111]:
def get_tokens_amount_for_position_list(positions, tick, vol_change, price0, price1): # pool_balance_0
    token0_total_amount = 0
    token1_total_amount = 0
    for position in positions:
        token0_amount, token1_amount = position.get_tokens_amount(tick, vol_change, price0, price1) #pool_balance_0
        token0_total_amount += token0_amount
        token1_total_amount += token1_amount
    return token0_total_amount, token1_total_amount

In [112]:
class Hedge:
    def __init__(self, init_balance, fee_rate):
        self.balance = init_balance
        self.fee_rate = fee_rate
        self.entry_price = 0
        self.amount = 0
        
    def set_hedge_amount(self, new_amount, price): #new_amount - all token in all positions in pool
        amount_change = new_amount - self.amount
        if amount_change == 0:
            return 0
        if amount_change > 0:
            self._update_entry_price(amount_change, price)
        else:
            self.balance += amount_change * (price - self.entry_price)
        self.amount = new_amount
        fee = abs(amount_change) * price * self.fee_rate 
        self.balance -= fee # to account for the gas used
        return fee
    
    def _update_entry_price(self, amount_change, price):
        self.entry_price = (self.amount * self.entry_price + amount_change * price) / (self.amount + amount_change)
    
    def get_unrealized_profit(self, price):
        return -self.amount * (price - self.entry_price)
    
    def get_margin_balance(self, price):
        return self.balance + self.get_unrealized_profit(price)
    
    def pay_funding(self, price, funding_rate):
        funding = self.amount * price * funding_rate
        self.balance += funding
        return funding

In [113]:
def calculate_liquidity(X_amount, Y_amount, current_tick, tick_lower, tick_upper):
    tick_base = 1.0001
    sqrt_price = tick_base ** (current_tick / 2)
    sqrt_price_l = tick_base ** (tick_lower / 2)
    sqrt_price_u = tick_base ** (tick_upper / 2)
    
    if current_tick <= tick_lower:
        liq_x = (X_amount / (1 / sqrt_price_l - 1 / sqrt_price_u))
        liq_y = liq_x
    elif current_tick >= tick_upper:
        liq_y = (Y_amount / (sqrt_price_u - sqrt_price_l))
        liq_x = liq_y
    else:
        liq_x = (X_amount / (1 / sqrt_price - 1 / sqrt_price_u))
        liq_y = (Y_amount / (sqrt_price - sqrt_price_l))
        
    return liq_x, liq_y

In [114]:
def get_pool_balance(amount0, amount1, price0, price1):
    return amount0 * price0 + amount1 * price1

In [115]:
market_data = all_data
market_data = market_data.iloc[1:]
market_data.index = market_data['datetime']
market_data = market_data.sort_index()

In [116]:
test_data = market_data.copy()
test_data['token1_p'] = 1
test_data['tick'] = np.log(market_data['token0_p']) / np.log(1.0001)

test_data['vol'] = (test_data['token0_p']/test_data['token0_p'].shift(1)).rolling(10).std() * 365 # expanding(30)
test_data['vol_change'] = test_data['vol']/test_data['vol'].shift(1).rolling(10).mean()-1
test_data['vol_change'] = test_data['vol_change'].fillna(1)

In [117]:
px.line(test_data, x='datetime', y=['vol'])

In [118]:
px.line(test_data, x='datetime', y=['vol_change'])

In [119]:
test_data

,datetime,token0-token1,token0_p,token1_p,token0_f,token1_f,tick,vol,vol_change
datetime,,,,,,,,,
2021-05-06 01:00:00,2021-05-06 01:00:00,3402.728895,3402.729189,1,NaN,NaN,81327.397012,NaN,1.000000
2021-05-06 02:00:00,2021-05-06 02:00:00,3496.202887,3496.203173,1,NaN,NaN,81598.408159,NaN,1.000000
2021-05-06 03:00:00,2021-05-06 03:00:00,3480.198291,3480.198579,1,0.000682,NaN,81552.523695,NaN,1.000000
2021-05-06 04:00:00,2021-05-06 04:00:00,3503.486592,3503.486877,1,NaN,NaN,81619.220712,NaN,1.000000
2021-05-06 05:00:00,2021-05-06 05:00:00,3487.077710,3487.077997,1,NaN,NaN,81572.272485,NaN,1.000000
...,...,...,...,...,...,...,...,...,...
2021-09-07 19:00:00,2021-09-07 19:00:00,3574.063969,3574.064249,1,0.000100,NaN,81818.677384,9.428332,1.099796
2021-09-07 20:00:00,2021-09-07 20:00:00,3419.354433,3419.354725,1,NaN,NaN,81376.139889,10.213492,0.933207
2021-09-07 21:00:00,2021-09-07 21:00:00,3451.361369,3451.361659,1,NaN,NaN,81469.314353,10.107135,0.641341


In [120]:
total_balance = 1000
position_width = 12000
hedge_balance = total_balance / 4
pool_balance = total_balance - hedge_balance

init_market_state = test_data.iloc[0]
init_tick = init_market_state['tick']
tick_lower = init_tick-position_width
tick_upper = init_tick+position_width
init_liquidity = min(calculate_liquidity(X_amount=pool_balance / 2 / init_market_state['token0_p'],
                                         Y_amount=pool_balance / 2 / init_market_state['token1_p'],
                                         current_tick=init_tick,
                                         tick_lower=tick_lower,
                                         tick_upper=tick_upper))

hedge0 = Hedge(init_balance=hedge_balance/2, fee_rate=0.0004)
hedge1 = Hedge(init_balance=hedge_balance/2, fee_rate=0.0004)

positions = [UniPosition2(init_liquidity=init_liquidity, init_pool_balance = pool_balance,init_tick=init_tick,init_position_width=position_width)]

In [121]:
result = []
for idx, (index, state) in tqdm(enumerate(test_data.iterrows())):
    tick = state['tick']
    vol_change = state['vol_change']
    price0 = state['token0_p']
    price1 = state['token1_p']
    funding_rate0 = state['token0_f']
    funding_rate1 = state['token1_f']
        
    pool_amount0, pool_amount1 = get_tokens_amount_for_position_list(positions, tick, vol_change, price0, price1) #pool_balance_0
    
    pool_balance = get_pool_balance(pool_amount0, pool_amount1, price0, price1)
    
    # rehedge condition
    fee0 = hedge0.set_hedge_amount(pool_amount0, price0)
    fee1 = hedge1.set_hedge_amount(pool_amount1, price1)
    
    # short with perpetuals
    if pd.notna(funding_rate0):
        funding0 = hedge0.pay_funding(price0, funding_rate0)
    else:
        funding0 = 0
    if pd.notna(funding_rate1):
        funding1 = hedge1.pay_funding(price1, funding_rate1)
    else:
        funding1 = 0
    
    margin_balance0 = hedge0.get_margin_balance(price0)
    margin_balance1 = hedge1.get_margin_balance(price1)
    
    state_result = dict(state)
    state_result.update({'pool_amount0': pool_amount0,
                         'pool_amount1': pool_amount1,
                         'pool_balance': pool_balance,
                         'margin_balance0': margin_balance0,
                         'margin_balance1': margin_balance1,
                         'fee0': fee0,
                         'fee1': fee1,
                         'funding0': funding0,
                         'funding1': funding1})
    result.append(state_result)

2999it [00:00, 13855.07it/s]


In [122]:
output = pd.DataFrame(result)
output['acc_f'] = (output['funding0'] + output['funding1']).cumsum()
output['IL'] = output['pool_balance'] + output['margin_balance0'] + output['margin_balance1'] - output['acc_f'] - total_balance
output['pnl_hedge'] = output['margin_balance0'] + output['margin_balance1'] - output['acc_f'] - hedge_balance
output

,datetime,token0-token1,token0_p,token1_p,token0_f,token1_f,tick,vol,vol_change,pool_amount0,...,pool_balance,margin_balance0,margin_balance1,fee0,fee1,funding0,funding1,acc_f,IL,pnl_hedge
0,2021-05-06 01:00:00,3402.728895,3402.729189,1,NaN,NaN,81327.397012,NaN,1.000000,0.110206,...,750.000000,124.850000,124.850000,0.150000,0.150000,0.000000,0,0.000000,-0.300000,-0.300000
1,2021-05-06 02:00:00,3496.202887,3496.203173,1,NaN,NaN,81598.408159,NaN,1.000000,0.106918,...,760.146676,114.544041,124.845464,0.004597,0.004536,0.000000,0,0.000000,-0.463819,-10.610495
2,2021-05-06 03:00:00,3480.198291,3480.198579,1,0.000682,NaN,81552.523695,NaN,1.000000,0.107472,...,758.431069,116.509682,124.844692,0.000770,0.000772,0.255229,0,0.255229,-0.469786,-8.900855
3,2021-05-06 04:00:00,3503.486592,3503.486877,1,NaN,NaN,81619.220712,NaN,1.000000,0.106668,...,760.924523,114.005723,124.843569,0.001127,0.001123,0.000000,0,0.255229,-0.481414,-11.405937
4,2021-05-06 05:00:00,3487.077710,3487.077997,1,NaN,NaN,81572.272485,NaN,1.000000,0.107233,...,759.169591,115.755230,124.842778,0.000789,0.000791,0.000000,0,0.255229,-0.487630,-9.657222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2994,2021-09-07 19:00:00,3574.063969,3574.064249,1,0.000100,NaN,81818.677384,9.428332,1.099796,0.104279,...,768.368127,13.698262,120.940668,0.005052,0.004978,0.037270,0,19.573454,-116.566397,-134.934524
2995,2021-09-07 20:00:00,3419.354433,3419.354725,1,NaN,NaN,81376.139889,10.213492,0.933207,0.109611,...,751.827280,29.823908,120.933212,0.007293,0.007456,0.000000,0,19.573454,-116.989054,-118.816334
2996,2021-09-07 21:00:00,3451.361369,3451.361659,1,NaN,NaN,81469.314353,10.107135,0.641341,0.108479,...,755.317429,26.314029,120.931656,0.001563,0.001556,0.000000,0,19.573454,-117.010340,-122.327769
2997,2021-09-07 22:00:00,3415.905886,3415.906179,1,NaN,NaN,81366.048925,10.096097,0.490175,0.109734,...,751.449069,30.158475,120.929931,0.001715,0.001724,0.000000,0,19.573454,-117.035979,-118.485048


In [123]:
px.line(output, x='datetime', y=['pool_balance','acc_f', 'pnl_hedge', 'IL'])

In [124]:
#px.line(output, x='datetime', y=['acc_f', 'pnl_hedge', 'IL']) #'pool_balance'